In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils import data
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch import nn


/var/folders/0c/y7bd5j0d2t556b_zf587bmsw0000gn/T/ipykernel_64723/4048784357.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#torch.set_default_device('mps')

In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


The Data set is the AutoMPG Dataset


In [10]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [11]:
dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


Now procede to clean the data by removing the Nan and fixing other values.


In [12]:
dataset.isna().sum()


MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [13]:
dataset = dataset.dropna()


Change categorical column


In [14]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
393,27.0,4,140.0,86.0,2790.0,15.6,82,False,False,True
394,44.0,4,97.0,52.0,2130.0,24.6,82,True,False,False
395,32.0,4,135.0,84.0,2295.0,11.6,82,False,False,True
396,28.0,4,120.0,79.0,2625.0,18.6,82,False,False,True
397,31.0,4,119.0,82.0,2720.0,19.4,82,False,False,True


In [15]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print("shape of train_dataset: ", train_dataset.shape)
print("shape of test_dataset: ", test_dataset.shape)


shape of train_dataset:  (314, 10)
shape of test_dataset:  (78, 10)


In [16]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')
print("shape of train_features: ", train_features.shape)
print("shape of test_features: ", test_features.shape)
print("shape of train_labels: ", train_labels.shape)
print("shape of test_labels: ", test_labels.shape)

shape of train_features:  (314, 9)
shape of test_features:  (78, 9)
shape of train_labels:  (314,)
shape of test_labels:  (78,)


In [18]:
batch = 10
X =np.array(train_features.values.astype(np.float32))
X = torch.Tensor(X)
y = np.array(train_labels.values.astype(np.float32))
y = torch.Tensor(y)
y = torch.reshape(y, (-1,1))
train = TensorDataset(X,y)
train_dataloader = DataLoader(train, batch_size=batch, shuffle=True)
sample = next(iter(train_dataloader))
print("sample batch: ", sample)
print("batch len: ", len(sample))

sample batch:  [tensor([[5.0000e+00, 1.8300e+02, 7.7000e+01, 3.5300e+03, 2.0100e+01, 7.9000e+01,
         1.0000e+00, 0.0000e+00, 0.0000e+00],
        [6.0000e+00, 2.5000e+02, 1.0500e+02, 3.8970e+03, 1.8500e+01, 7.5000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [6.0000e+00, 2.5000e+02, 7.8000e+01, 3.5740e+03, 2.1000e+01, 7.6000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [8.0000e+00, 3.1800e+02, 1.5000e+02, 4.4980e+03, 1.4500e+01, 7.5000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [4.0000e+00, 1.4000e+02, 8.8000e+01, 2.8700e+03, 1.8100e+01, 8.0000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [4.0000e+00, 9.7000e+01, 7.8000e+01, 2.1880e+03, 1.5800e+01, 8.0000e+01,
         1.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0000e+00, 7.9000e+01, 7.0000e+01, 2.0740e+03, 1.9500e+01, 7.1000e+01,
         1.0000e+00, 0.0000e+00, 0.0000e+00],
        [6.0000e+00, 1.7300e+02, 1.1500e+02, 2.5950e+03, 1.1300e+01, 7.9000e+01,
         0.000

In [20]:
for X, y in train_dataloader:
    print(f"Shape of X [B, F]: {X.shape}")
    print(f"Shape of y [B, L]: {y.shape} {y.dtype}")
    break

Shape of X [B, F]: torch.Size([10, 9])
Shape of y [B, L]: torch.Size([10, 1]) torch.float32


In [21]:
X =np.array(test_features.values.astype(np.float32))
X = torch.Tensor(X)
y = np.array(test_labels.values.astype(np.float32))
y = torch.Tensor(y)
y = torch.reshape(y, (-1,1))
test = TensorDataset(X,y)
test_dataloader = DataLoader(test, batch_size=batch, shuffle=True)
sample = next(iter(test_dataloader))
print("sample batch: ", sample[0])
print("batch len: ", len(sample))

sample batch:  tensor([[4.0000e+00, 1.0500e+02, 7.4000e+01, 2.1900e+03, 1.4200e+01, 8.1000e+01,
         1.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0000e+00, 9.7000e+01, 7.5000e+01, 2.1710e+03, 1.6000e+01, 7.5000e+01,
         0.0000e+00, 1.0000e+00, 0.0000e+00],
        [4.0000e+00, 1.2200e+02, 8.0000e+01, 2.4510e+03, 1.6500e+01, 7.4000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [8.0000e+00, 3.5000e+02, 1.0500e+02, 3.7250e+03, 1.9000e+01, 8.1000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [4.0000e+00, 9.8000e+01, 6.6000e+01, 1.8000e+03, 1.4400e+01, 7.8000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [6.0000e+00, 2.5000e+02, 8.8000e+01, 3.0210e+03, 1.6500e+01, 7.3000e+01,
         0.0000e+00, 0.0000e+00, 1.0000e+00],
        [6.0000e+00, 1.6300e+02, 1.2500e+02, 3.1400e+03, 1.3600e+01, 7.8000e+01,
         1.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0000e+00, 1.5100e+02, 8.4000e+01, 2.6350e+03, 1.6400e+01, 8.1000e+01,
         0.0000

In [23]:
for X, y in test_dataloader:
    print(f"Shape of X [B, F]: {X.shape}")
    print(f"Shape of y [B, L]: {y.shape} {y.dtype}")
    break

Shape of X [B, F]: torch.Size([10, 9])
Shape of y [B, L]: torch.Size([10, 1]) torch.float32


In [24]:
class NNModel(nn.Module):
    def __init__(self):
        #super(NNModel, self).__init__()
        super().__init__()
        self.norm = nn.BatchNorm1d(9)
        self.fc1 = nn.Linear(9,1)


    def forward(self, x):
        t = self.norm(x)
        t = self.fc1(t)
        return t


In [25]:
class NNModel2(nn.Module):
    def __init__(self):
        #super(NNModel, self).__init__()
        super().__init__()
        self.norm = nn.BatchNorm1d(9)
        self.fc1 = nn.Linear(9,18)
        self.fc2 = nn.Linear(18,9)
        self.fc3 = nn.Linear(9,1)
        self.relu = nn.ReLU()


    def forward(self, x):
        t = self.norm(x)
        t = self.fc1(t)
        t = self.relu(t)
        t = self.fc2(t)
        t = self.relu(t)
        t = self.fc3(t)
        return t

In [26]:
class NNModel3(nn.Module):
    def __init__(self):
        #super(NNModel, self).__init__()
        super().__init__()
        self.norm = nn.BatchNorm1d(9)
        self.fc1 = nn.Linear(9,18)
        self.fc2 = nn.Linear(18,18)
        self.fc3 = nn.Linear(18,9)
        self.fc4 = nn.Linear(9,1)
        self.relu = nn.ReLU()


    def forward(self, x):
        t = self.norm(x)
        t = self.fc1(t)
        t = self.relu(t)
        t = self.fc2(t)
        t = self.relu(t)
        t = self.fc3(t)
        t = self.relu(t)
        t = self.fc4(t)
        return t

In [51]:
#nnet = NNModel()
#nnet = NNModel2()
nnet = NNModel3()

In [52]:
from torchinfo import summary
batch = 10
summary(nnet, input_size=(batch, 9), device='cpu', col_names=['input_size', 'output_size',
                                                              'num_params'])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NNModel3                                 [10, 9]                   [10, 1]                   --
├─BatchNorm1d: 1-1                       [10, 9]                   [10, 9]                   18
├─Linear: 1-2                            [10, 9]                   [10, 18]                  180
├─ReLU: 1-3                              [10, 18]                  [10, 18]                  --
├─Linear: 1-4                            [10, 18]                  [10, 18]                  342
├─ReLU: 1-5                              [10, 18]                  [10, 18]                  --
├─Linear: 1-6                            [10, 18]                  [10, 9]                   171
├─ReLU: 1-7                              [10, 9]                   [10, 9]                   --
├─Linear: 1-8                            [10, 9]                   [10, 1]                   10
Total params: 721
Trainable para

In [53]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    print("size: ", size)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #print("X: " , X)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [54]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    print("size: ", size)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [55]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(nnet.parameters(), lr=0.001)


In [56]:
device = 'cpu'
device = mps_device
nnet = nnet.to(device=device)
epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, nnet, criterion, optimizer, device)
    test(test_dataloader, nnet, criterion, device)
print("Done!")

Epoch 1
-------------------------------
size:  314
Train loss: 487.735504  [   10/  314]
Train loss: 646.967712  [  110/  314]
Train loss: 733.222412  [  210/  314]
Train loss: 559.697937  [  310/  314]
size:  78
Test Error: 
 Accuracy: 0.0%, Avg loss: 626.324604 

Epoch 2
-------------------------------
size:  314
Train loss: 508.192291  [   10/  314]
Train loss: 610.192200  [  110/  314]
Train loss: 581.190735  [  210/  314]
Train loss: 532.875916  [  310/  314]
size:  78
Test Error: 
 Accuracy: 0.0%, Avg loss: 598.704868 

Epoch 3
-------------------------------
size:  314
Train loss: 561.023132  [   10/  314]
Train loss: 499.153046  [  110/  314]
Train loss: 414.193695  [  210/  314]
Train loss: 657.088989  [  310/  314]
size:  78
Test Error: 
 Accuracy: 0.0%, Avg loss: 538.398800 

Epoch 4
-------------------------------
size:  314
Train loss: 417.561859  [   10/  314]
Train loss: 556.237976  [  110/  314]
Train loss: 535.646851  [  210/  314]
Train loss: 383.425629  [  310/  314]

In [44]:
sample = next(iter(test_dataloader))
Y = nnet(sample[0].to(device=device))
print("Real labels: ", sample[1])
print("Predicted labels: ", Y)
print("Demo done!")

Real labels:  tensor([[43.1000],
        [15.0000],
        [32.0000],
        [18.0000],
        [33.0000],
        [31.9000],
        [28.1000],
        [15.0000],
        [20.0000],
        [13.0000]])
Predicted labels:  tensor([[32.8017],
        [14.9792],
        [38.5372],
        [18.7852],
        [33.4279],
        [32.1571],
        [29.9590],
        [15.5542],
        [19.9970],
        [16.9723]], device='mps:0', grad_fn=<LinearBackward0>)
Demo done!
